## <center> Аттестация. Практическое задание в Jupiter

В данном блоке вам предстоит решить задачу, которую автор когда-то решал на одном из своих собеседований.
Вам предоставлена таблица с данными (скачать файл можно на платформе) о ежедневной выработке энергии солнечными батареями. Информация в ней содержится в двух столбцах: 
* date - дата наблюдения
* cum_power - накопленная (кумулятивная) выработка энергии на указанный день

То есть перед вами классический пример временного ряда: числовой показатель, зависящий от времени. Давайте с ним поработаем.


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
solar_data = pd.read_csv('data/solarpower.csv')
solar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664 entries, 0 to 663
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       664 non-null    object 
 1   cum_power  664 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.5+ KB


1. В нашем временном ряде есть несколько пропусков, они обозначены числом -1. Заполните пропуски средним арифметическим между двумя соседними наблюдениями: </p>
    $$x^*_i  = \frac{x_{i-1} - x_{i+1}}{2}$$ 
    
Совет: проще всего будет сделать это, используя цикл по индексам элементов столбца cum_power, но вы можете предложить более изящное решение.


In [3]:
# смотрим на пропуски
display(solar_data.iloc[[44, 45, 46, 646, 647, 648]]) 

# выполняем преобразования
indexes = solar_data.index[solar_data['cum_power'] == -1]
for i in indexes:
    solar_data.iloc[i,1] = (solar_data.iloc[i-1,1] + solar_data.iloc[i+1,1]) / 2 # для граничных значений данные возьмутся с другого конца Series

# смотрим результат
display(solar_data.iloc[[44, 45, 46, 646, 647, 648]]) 

,date,cum_power
44,2017-02-14,20248.0
45,2017-02-15,-1.0
46,2017-02-16,20262.0
646,2018-10-09,27986.0
647,2018-10-10,-1.0
648,2018-10-11,28010.0


,date,cum_power
44,2017-02-14,20248.0
45,2017-02-15,20255.0
46,2017-02-16,20262.0
646,2018-10-09,27986.0
647,2018-10-10,27998.0
648,2018-10-11,28010.0


2. Теперь, когда таблица заполнена, создайте столбец day_power - ежедневный прирост выработки солнечной энергии.

    После чего создайте данные о ежемесячной выработке энергии на каждый год. 
    
Совет: для того, чтобы выделить из даты год и месяц проще всего воспользоваться методом аксессора dt.to_period('M'), который позволяет выделять периоды, в данном случае 'М' - месяц.

In [4]:
# ежедневный прирост выработки солнечной энергии
solar_data['day_power'] = solar_data['cum_power'].diff()
solar_data

,date,cum_power,day_power
0,2017-01-01,20111.0,NaN
1,2017-01-02,20112.0,1.0
2,2017-01-03,20115.0,3.0
3,2017-01-04,20117.0,2.0
4,2017-01-05,20119.0,2.0
...,...,...,...
659,2018-10-22,28101.0,6.0
660,2018-10-23,28109.0,8.0
661,2018-10-24,28115.0,6.0
662,2018-10-25,28117.0,2.0


In [5]:
# ежемесячная выработка энергии на каждый год
solar_data['date'] = pd.to_datetime(solar_data['date'])
solar_data['month'] = solar_data['date'].dt.to_period('M')

month_power = solar_data.groupby(by='month')['day_power'].sum()

month_power

month
2017-01     97.0
2017-02    104.0
2017-03    334.0
2017-04    489.0
2017-05    589.0
2017-06    645.0
2017-07    562.0
2017-08    467.0
2017-09    346.0
2017-10    199.0
2017-11     97.0
2017-12     36.0
2018-01     64.0
2018-02    194.0
2018-03    270.0
2018-04    439.0
2018-05    652.0
2018-06    563.0
2018-07    682.0
2018-08    530.0
2018-09    416.0
2018-10    234.0
Freq: M, Name: day_power, dtype: float64

3. Постройте график временного ряда - ежемесячной выработки солнечной энергии. Не забудьте подписать оси и сам график.
Укажите в какие сезоны наблюдается подъем, а в какие спад?

In [6]:
fig = px.bar(x=month_power.index.to_timestamp(), y=month_power, title='Ежемесячная выработка солнечной энергии')
fig.update_xaxes(title_text='Месяц выработки')
fig.update_yaxes(title_text='Количество выработанной энергии')
fig.show()

### Вывод
На графике мы видим бимодальное распределение значений выработки солнечной энергии. Но так как мы рассматриваем распределение за 2 года, то такое поведение ожидаемо. По итогу получаем пик выработки энергии на летний сезон каждого года и минимальную выработку в зимние месяцы.
Также на графике видно падение выработки энергии в июне 2018. Если дополнительно уточнить место установки солнечных батарей и посмотреть прогноз погоды на эти даты, то скорее всего мы увидим гораздо более облачную погоду, чем в соседние месяцы.